In [ ]:
#Notebook description

#This notebook is being used to evaluate the techinical market conditions of a single asset and assess
#the appropriate strategy to take in order to maximize returns.

In [ ]:
#take all compuation functions and put them in a separate file

#simplify the date x axis on the percent drawdown chart

#default the zoom range to a comfortable range, and create a dropdown to select the time range for Volatility section

#properly label and annoate the garch models

#Remove the VIX charting, its redudnant now that we have the volatility models

In [ ]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")
# Load libraries
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation, Helper, Models
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()
helper = Helper()
model = Models()


In [ ]:
#PARAMETERS (Adjust parmeters here)
# Define parameters for the analysis (Adjust these as needed)
interval = '1d'
period     = '10y'
risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days
#should be a string or None
ticker_str = 'ETH-USD' #ticker to analyze
primary_benchmark_str = 'BTC-USD'# #primary benchmark to compare against
secondary_benchmark_str = None # #secondary benchmark to compare against
length_of_plots = 10 #number of years of data to plot (after computing the period, this will be adjusted to the closest available data)
trading_strategy = 'position'  # Options: 'position' or 'swing', # Determines the trading strategy to use for time frames

In [ ]:
# Load data
# Define timeframe lists for different strategies
position_time_frames = [21, 50, 200]  # For position trading (longer term)
swing_time_frames = [3, 9, 21]        # For swing trading (shorter term)

# Populate the time frame variables based on selected strategy
if trading_strategy == "position":
    time_frame_short = position_time_frames[0]
    time_frame_mid = position_time_frames[1]
    time_frame_long = position_time_frames[2]
else:  # swing trading
    time_frame_short = swing_time_frames[0]
    time_frame_mid = swing_time_frames[1]
    time_frame_long = swing_time_frames[2]
    

ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
vix = yf.Ticker('^VIX').history(period=period, interval=interval)

# Fetch and simplify benchmark data only if the strings are not None
if primary_benchmark_str is not None:
    primary_benchmark = yf.Ticker(primary_benchmark_str).history(period=period, interval=interval)
    primary_benchmark = helper.simplify_datetime_index(primary_benchmark)
    (primary_benchmark)
else:
    primary_benchmark = None

if secondary_benchmark_str is not None:
    secondary_benchmark = yf.Ticker(secondary_benchmark_str).history(period=period, interval=interval)
    secondary_benchmark = helper.simplify_datetime_index(secondary_benchmark)
else:
    secondary_benchmark = None

# Simplify ticker and vix
ticker = helper.simplify_datetime_index(ticker)
vix = helper.simplify_datetime_index(vix)

# Align ticker data with primary benchmark if available
if primary_benchmark is not None:
    ticker = ticker[ticker.index.isin(primary_benchmark.index)]

# Calculate ticker returns
ticker_monthly_returns = qc.calculate_returns(ticker, frequency='monthly')
ticker_weekly_returns = qc.calculate_returns(ticker, frequency='weekly')
ticker_daily_returns = qc.calculate_returns(ticker, frequency='daily')

# Calculate primary benchmark returns if it exists
if primary_benchmark is not None:
    primary_benchmark_monthly_returns = qc.calculate_returns(primary_benchmark, frequency='monthly')
    primary_benchmark_weekly_returns = qc.calculate_returns(primary_benchmark, frequency='weekly')
    primary_benchmark_daily_returns = qc.calculate_returns(primary_benchmark, frequency='daily')

# Calculate secondary benchmark returns if it exists
if secondary_benchmark is not None:
    secondary_benchmark_monthly_returns = qc.calculate_returns(secondary_benchmark, frequency='monthly')
    secondary_benchmark_weekly_returns = qc.calculate_returns(secondary_benchmark, frequency='weekly')
    secondary_benchmark_daily_returns = qc.calculate_returns(secondary_benchmark, frequency='daily')

# Calculate VIX returns
vix_monthly_returns = qc.calculate_returns(vix, frequency='monthly')
vix_weekly_returns = qc.calculate_returns(vix, frequency='weekly')
vix_daily_returns = qc.calculate_returns(vix, frequency='daily')


In [ ]:
#REGIME CHANGES: Candlestick with RSI and Bollinger Bands
qp.plot_candlestick(ticker, period='2y', bollinger_window=50, title="Candlestick with 50-Period Bollinger Bands")
qp.plot_candlestick(ticker, period='5y', bollinger_window=200, title="Candlestick with 200-Period Bollinger Bands ")

In [ ]:
#REGIME CHANGES: Percentage Drop from Highest Peak
n = int(252 / 2)
qp.plot_percentage_drop(ticker, n=n, title=f'{ticker_str} Percentage Drop from Highest Peak')

In [ ]:
#Volatility: VIX FIX 

ticker_vix_fix = qc.vix_fix(ticker)

if primary_benchmark_str is not None:
    primary_benchmark_vix_fix = qc.vix_fix(primary_benchmark)
if secondary_benchmark_str is not None:
    secondary_benchmark_vix_fix = qc.vix_fix(secondary_benchmark)


qp.plot_series_with_stdev_bands(
    ticker_vix_fix,
    title='VIX Fix with Mean and Standard Deviations',
    stdev_values=[-0.5, 0.5, 1.5, 3]
)


In [ ]:
#Seasonality: Monthly, Weekly, and Daily Returns
#SEASONALITY: Monthly Seasonality
fig_ticker_Seasonality_Monthly = qp.plot_seasonality(ticker_monthly_returns, f'Monthly Seasonality: {ticker_str}', 'monthly')
fig_ticker_Seasonality_Monthly.show()

#SEASONALITY: Weekly Seasonality
fig_ticker_Seasonality_weekly = qp.plot_seasonality(ticker_weekly_returns, f'Weekly Seasonality: {ticker_str}', 'weekly')
fig_ticker_Seasonality_weekly.show()

#SEASONALITY: Daily Seasonality
fig_ticker_Seasonality_daily = qp.plot_seasonality(ticker_daily_returns, f'Daily Seasonality: {ticker_str}', 'daily')
fig_ticker_Seasonality_daily.show()

In [ ]:
#Calculations

sharpe_ticker_short = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_short])[f'sharpe_ratio_{time_frame_short}']
sharpe_ticker_mid   = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_mid])[f'sharpe_ratio_{time_frame_mid}']
sharpe_ticker_long  = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_long])[f'sharpe_ratio_{time_frame_long}']


if primary_benchmark is not None:
    spread_short_primary = primary_benchmark['Close'].pct_change(time_frame_short) - ticker['Close'].pct_change(time_frame_short)
    spread_mid_primary = primary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_primary = primary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_short_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_short]
    )[f'sharpe_ratio_{time_frame_short}']
    
    sharpe_benchmark_mid_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )[f'sharpe_ratio_{time_frame_mid}']
    
    sharpe_benchmark_long_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )[f'sharpe_ratio_{time_frame_long}']
    
    sharpe_spread_short_primary = sharpe_benchmark_short_primary - sharpe_ticker_short
    sharpe_spread_mid_primary = sharpe_benchmark_mid_primary - sharpe_ticker_mid
    sharpe_spread_long_primary = sharpe_benchmark_long_primary - sharpe_ticker_long

# Calculate spreads and Sharpe ratios for the secondary benchmark if it exists
if secondary_benchmark is not None:
    spread_short_secondary = secondary_benchmark['Close'].pct_change(time_frame_short) - ticker['Close'].pct_change(time_frame_short)
    spread_mid_secondary = secondary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_secondary = secondary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_short_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_short]
    )[f'sharpe_ratio_{time_frame_short}']
    
    sharpe_benchmark_mid_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )[f'sharpe_ratio_{time_frame_mid}']
    
    sharpe_benchmark_long_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )[f'sharpe_ratio_{time_frame_long}']
    
    sharpe_spread_short_secondary = sharpe_benchmark_short_secondary - sharpe_ticker_short
    sharpe_spread_mid_secondary = sharpe_benchmark_mid_secondary - sharpe_ticker_mid
    sharpe_spread_long_secondary = sharpe_benchmark_long_secondary - sharpe_ticker_long



In [ ]:
# ...existing code...

# Combine all risk-adjusted return plots into a single loop for short, mid, and long term

# Prepare benchmark lists for each term
benchmarks_short = []
if primary_benchmark is not None:
    benchmarks_short.append((primary_benchmark, primary_benchmark_str, sharpe_spread_short_primary, time_frame_short, length_of_plots))
if secondary_benchmark is not None:
    benchmarks_short.append((secondary_benchmark, secondary_benchmark_str, sharpe_spread_short_secondary, time_frame_short, length_of_plots))

benchmarks_mid = []
if primary_benchmark is not None:
    benchmarks_mid.append((primary_benchmark, primary_benchmark_str, sharpe_spread_mid_primary, time_frame_mid, length_of_plots))
if secondary_benchmark is not None:
    benchmarks_mid.append((secondary_benchmark, secondary_benchmark_str, sharpe_spread_mid_secondary, time_frame_mid, length_of_plots))

benchmarks_long = []
if primary_benchmark is not None:
    benchmarks_long.append((primary_benchmark, primary_benchmark_str, sharpe_spread_long_primary, time_frame_long, length_of_plots))
if secondary_benchmark is not None:
    benchmarks_long.append((secondary_benchmark, secondary_benchmark_str, sharpe_spread_long_secondary, time_frame_long, length_of_plots))

# Combine all benchmarks into one list with a label for the term
all_benchmarks = (
    [("Short Term",) + b for b in benchmarks_short] +
    [("Mid Term",) + b for b in benchmarks_mid] +
    [("Long Term",) + b for b in benchmarks_long]
)

for term, benchmark, benchmark_str, sharpe_spread, time_frame, default_years in all_benchmarks:
    qp.create_spread_plot(
        sharpe_spread,
        title=f'{time_frame} Day: Benchmark ({benchmark_str}) minus {ticker_str} (Sharpe) [{term}]',
        default_years=default_years
    ).show()

    qp.plot_returns(
        ticker['Close'],
        benchmark_series=benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame} days [{term}]',
        window=time_frame,
        default_years=default_years
    ).show()
# ...existing code...

In [ ]:

#Fama-French Factor Analysis: Calculations
import yfinance as yf
import pandas as pd
import pandas as pd

# Define period & interval
period = "10y"
interval = "1d"

# Download Fama-French factors and asset price data
tickers = ["SPY", "SIZE", "VLUE", "QUAL", "USMV", "MTUM", "BIL"]
if primary_benchmark_str is not None:
    tickers.append(primary_benchmark_str)
if secondary_benchmark_str is not None:
    tickers.append(secondary_benchmark_str)
    
prices = {ticker: yf.Ticker(ticker).history(period=period, interval=interval)['Close'] for ticker in tickers}
prices_df = pd.DataFrame(prices)

# Calculate daily returns
returns = prices_df.pct_change().dropna()

# Calculate excess market return (Market - Risk-Free)
factor_returns = pd.DataFrame()
factor_returns['Mkt-RF'] = returns["SPY"] - returns["BIL"]
factor_returns['SMB'] = (returns["SIZE"] - returns["SPY"]) / 2
factor_returns['HML'] = (returns["VLUE"] - returns["QUAL"]) / 2
factor_returns['RMW'] = (returns["QUAL"] - returns["VLUE"]) / 2
factor_returns['CMA'] = (returns["USMV"] - returns["MTUM"]) / 2
rf_series = returns["BIL"]

#add the primary benchmark as a factor

if primary_benchmark_str is not None and primary_benchmark_str != 'SPY':
    factor_returns[primary_benchmark_str] = returns[primary_benchmark_str] - returns["BIL"]

if secondary_benchmark_str is not None and secondary_benchmark_str != 'SPY':
    factor_returns[secondary_benchmark_str] = returns[secondary_benchmark_str] - returns["BIL"]
    

factor_returns_capm = factor_returns[['Mkt-RF']].copy()
factor_returns_ff3 = factor_returns[['Mkt-RF', 'SMB', 'HML']].copy()
factor_returns_ff5 = factor_returns[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].copy()
#primary_benchmark_str might exist, secondary_benchmark_str might exist, make sure to add them as factors
custom_list = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

if primary_benchmark_str is not None and primary_benchmark_str != 'SPY':
    custom_list.append(primary_benchmark_str)
if secondary_benchmark_str is not None and secondary_benchmark_str != 'SPY':
    custom_list.append(secondary_benchmark_str)
    
factor_returns_ff5_custom = factor_returns[custom_list].copy()

data = yf.Ticker(ticker_str).history(period=period, interval=interval)

rolling_results_ff5_custom           = model.rolling_regression(data, rf_series, factor_returns_ff5_custom, window=252)


rolling_results_ff5_custom


In [ ]:
#Plotting the Fama-French Factor Analysis Results
from plotly.subplots import make_subplots

qp.plot_rolling_regression(rolling_results_ff5_custom, ticker_str, factor_returns_ff5_custom)